This is an attempt to infuse the data with another data from same domain i.e. infusing the common lit data with data from the [CBT dataset](https://huggingface.co/datasets/cbt).

the idea is that since the CBT also contains data from childrens book so the language model trained with this data would be like a child from grade 3-12 . 

In [1]:
!pip install fastcore

In [2]:
!yes Y | conda install -c huggingface -c conda-forge datasets

Solving environment: | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | 

In [3]:
from typing import List
from itertools import chain
from datasets import load_dataset
from fastcore.all import *
from pandas.core.frame import DataFrame
import numpy as np
import pandas as pd

## Read the data

In [4]:
dataset = load_dataset("cbt", "CN")

Dataset cbt downloaded and prepared to /root/.cache/huggingface/datasets/cbt/CN/1.1.0/bc79b04ef86e76912deb17112c013806daa07ef250fa8d4f9bd5cf490f02f80f. Subsequent calls will reuse this data.


In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['sentences', 'question', 'answer', 'options'],
        num_rows: 120769
    })
    test: Dataset({
        features: ['sentences', 'question', 'answer', 'options'],
        num_rows: 2500
    })
    validation: Dataset({
        features: ['sentences', 'question', 'answer', 'options'],
        num_rows: 2000
    })
})

Getting the sentences(20 sentences from each book).

In [6]:
dataset.column_names

{'train': ['sentences', 'question', 'answer', 'options'],
 'test': ['sentences', 'question', 'answer', 'options'],
 'validation': ['sentences', 'question', 'answer', 'options']}

In [7]:
train = L(dataset['train']['sentences'][:2])

In [8]:
train

(#2) [['With almost everything else to make them happy , they wanted one thing : they had no children .', 'This vexed the king even more than the queen , who was very clever and learned , and who had hated dolls when she was a child .', 'However , she , too in spite of all the books she read and all the pictures she painted , would have been glad enough to be the mother of a little prince .', 'The king was anxious to consult the fairies , but the queen would not hear of such a thing .', 'She did not believe in fairies : she said that they had never existed ; and that she maintained , though The History of the Royal Family was full of chapters about nothing else .', 'Well , at long and at last they had a little boy , who was generally regarded as the finest baby that had ever been seen .', 'Even her majesty herself remarked that , though she could never believe all the courtiers told her , yet he certainly was a fine child -- a very fine child .', 'Now , the time drew near for the chris

## Preprocessing
Initially the sentences are present as list of lists.
joining the sentences in each inner list into a bigger string as this is what will be fed to teh language model

In [9]:
def list2str(x):    return ''.join(x)

In [10]:
train.map?

In [11]:
train.map(list2str)

(#2) ["With almost everything else to make them happy , they wanted one thing : they had no children .This vexed the king even more than the queen , who was very clever and learned , and who had hated dolls when she was a child .However , she , too in spite of all the books she read and all the pictures she painted , would have been glad enough to be the mother of a little prince .The king was anxious to consult the fairies , but the queen would not hear of such a thing .She did not believe in fairies : she said that they had never existed ; and that she maintained , though The History of the Royal Family was full of chapters about nothing else .Well , at long and at last they had a little boy , who was generally regarded as the finest baby that had ever been seen .Even her majesty herself remarked that , though she could never believe all the courtiers told her , yet he certainly was a fine child -- a very fine child .Now , the time drew near for the christening party , and the king a

In [12]:
train = L(dataset['train']['sentences'])
test = L(dataset['test']['sentences'])
validation = L(dataset['validation']['sentences'])

In [13]:
train_processed = list(train.map(list2str))
test_processed = list(test.map(list2str))
valid_processed = list(test.map(list2str))

In [14]:
train_processed[:2]

["With almost everything else to make them happy , they wanted one thing : they had no children .This vexed the king even more than the queen , who was very clever and learned , and who had hated dolls when she was a child .However , she , too in spite of all the books she read and all the pictures she painted , would have been glad enough to be the mother of a little prince .The king was anxious to consult the fairies , but the queen would not hear of such a thing .She did not believe in fairies : she said that they had never existed ; and that she maintained , though The History of the Royal Family was full of chapters about nothing else .Well , at long and at last they had a little boy , who was generally regarded as the finest baby that had ever been seen .Even her majesty herself remarked that , though she could never believe all the courtiers told her , yet he certainly was a fine child -- a very fine child .Now , the time drew near for the christening party , and the king and qu

joining the lists into a single dataframe

In [15]:
len(train_processed)

120769

In [16]:
#big_list = list(chain(train_processed, test_processed, valid_processed))

In [17]:
#len(big_list)

In [18]:
df_cbt_train = pd.DataFrame(train_processed, columns=['excerpt'])
df_cbt_train['target'] = np.nan
df_cbt_test = pd.DataFrame(test_processed, columns=['excerpt'])
df_cbt_test['target'] = np.nan
df_cbt_valid = pd.DataFrame(valid_processed, columns=['excerpt'])
df_cbt_valid['target'] = np.nan

In [19]:
#df_cbt.shape

In [20]:
df_cbt_train.head(2)

,excerpt,target
0,With almost everything else to make them happy...,NaN
1,With almost everything else to make them happy...,NaN


## Pre-processing commonlit data prior to concatenation

In [21]:
df_aug_comlit = pd.read_csv('../input/common-lit-datset-with-synonym-replacement/aug_df.csv')
df_train_vanilla_comlit = pd.read_csv('../input/commonlitreadabilityprize/train.csv')
df_test_vanilla_comlit = pd.read_csv('../input/commonlitreadabilityprize/test.csv')

functions to drop and concatenate

In [22]:
def drop_cols(df: DataFrame, cols: List) -> DataFrame:
    """drop the given list of columns from the dataframe"""
    return df.drop(cols, axis=1)

def concat_dfs(dfs: List) -> DataFrame:
    """concatenate two dataframes"""
    return pd.concat(dfs)

In [23]:
df_aug_comlit = drop_cols(df_aug_comlit, ['Unnamed: 0', 
                                                  'id',
                                                  'url_legal',
                                                  'license', 
                                                  'standard_error'])

In [24]:
df_train_vanilla_comlit = drop_cols(df_train_vanilla_comlit, ['id',
                                                              'url_legal',
                                                              'license', 
                                                              'standard_error'])

In [25]:
df_aug_comlit.head(1)

,excerpt,target
0,When the young people returned to the ballroom...,-0.340259


In [26]:
df_train_vanilla_comlit.head(1)

,excerpt,target
0,When the young people returned to the ballroom...,-0.340259


## Infusing the CBT excerpt with the commonlit excerpt
* Preparing two versions-->
  * with augmented commonlit
  * with train set of vanilla commonlit

In [27]:
df_aug_comlit_infused_with_cbt = concat_dfs([df_aug_comlit, df_cbt_train])
df_vanilla_comlit_train_infused_with_cbt = concat_dfs([df_train_vanilla_comlit, df_cbt_train])

In [28]:
df_aug_comlit_infused_with_cbt.shape, df_vanilla_comlit_train_infused_with_cbt.shape

((128178, 2), (123603, 2))

In [29]:
df_aug_comlit_infused_with_cbt.head(3)

,excerpt,target
0,When the young people returned to the ballroom...,-0.340259
1,"All through dinner time, Mrs. Fayre was somewh...",-0.315372
2,"As Roger had predicted, the snow departed as q...",-0.580118


In [30]:
df_aug_comlit_infused_with_cbt.tail()

,excerpt,target
120764,"Hans Van Ripper as executor of his estate , ex...",NaN
120765,"Hans Van Ripper as executor of his estate , ex...",NaN
120766,"Whatever money the schoolmaster possessed , an...",NaN
120767,The mysterious event caused much speculation a...,NaN
120768,Knots of gazers and gossips were collected in ...,NaN


In [31]:
df_vanilla_comlit_train_infused_with_cbt.head(3)

,excerpt,target
0,When the young people returned to the ballroom...,-0.340259
1,"All through dinner time, Mrs. Fayre was somewh...",-0.315372
2,"As Roger had predicted, the snow departed as q...",-0.580118


## saving both the dataframes

In [32]:
df_aug_comlit_infused_with_cbt.to_csv('aug_comlit_infused_with_cbt.csv')
df_vanilla_comlit_train_infused_with_cbt.to_csv('vanilla_comlit_train_infused_with_cbt.csv')
df_cbt_train.to_csv('cbt_train.csv')
df_cbt_test.to_csv('cbt_test.csv')
df_cbt_valid.to_csv('cbt_valid.csv')